In [2]:
INPUT_MODEL_PATH = "../../models/calorie_regression_mobilenet.pt"
ONNX_MODEL_PATH = "../../models/calorie_regression_mobilenet.onnx"
COREML_MODEL_PATH = "../../models/calorie_regression_mobilenet.mlmodel"
IMAGE_SIZE = 224

## Imports and Extensions

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
%load_ext lab_black

In [5]:
import sys

sys.path.insert(0, "../..")

In [6]:
import torch
from torchvision import models
from seefood.model import CalorieNet
from onnx_coreml import convert
import coremltools

## Load Model

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torch.load(INPUT_MODEL_PATH).to(device)  ## TODO load state dict instead

## Convert PyTorch to ONNX

In [8]:
dummy_input = torch.rand(1, 3, IMAGE_SIZE, IMAGE_SIZE).to(device)

In [9]:
input_names = ["image"]
output_names = ["calories"]

In [11]:
torch.onnx.export(
    model,
    dummy_input,
    ONNX_MODEL_PATH,
    verbose=True,
    input_names=input_names,
    output_names=output_names,
)

graph(%image : Float(1, 3, 224, 224),
      %features.0.0.weight : Float(32, 3, 3, 3),
      %features.0.1.weight : Float(32),
      %features.0.1.bias : Float(32),
      %features.0.1.running_mean : Float(32),
      %features.0.1.running_var : Float(32),
      %features.0.1.num_batches_tracked : Long(),
      %features.1.conv.0.0.weight : Float(32, 1, 3, 3),
      %features.1.conv.0.1.weight : Float(32),
      %features.1.conv.0.1.bias : Float(32),
      %features.1.conv.0.1.running_mean : Float(32),
      %features.1.conv.0.1.running_var : Float(32),
      %features.1.conv.0.1.num_batches_tracked : Long(),
      %features.1.conv.1.weight : Float(16, 32, 1, 1),
      %features.1.conv.2.weight : Float(16),
      %features.1.conv.2.bias : Float(16),
      %features.1.conv.2.running_mean : Float(16),
      %features.1.conv.2.running_var : Float(16),
      %features.1.conv.2.num_batches_tracked : Long(),
      %features.2.conv.0.0.weight : Float(96, 16, 1, 1),
      %features.2.conv.0.1.w

## Convert ONNX to CoreML

In [ ]:
core_mlmodel = convert(model=ONNX_MODEL_PATH, image_input_names=["image"])

In [ ]:
core_mlmodel.save(COREML_MODEL_PATH)

## Model Evaluation

In [10]:
eval_model = coremltools.models.MLModel(COREML_MODEL_PATH)

In [11]:
eval_model

input {
  name: "image"
  type {
    imageType {
      width: 224
      height: 224
      colorSpace: RGB
    }
  }
}
output {
  name: "calories"
  type {
    multiArrayType {
      dataType: FLOAT32
    }
  }
}
metadata {
  userDefined {
    key: "coremltoolsVersion"
    value: "3.3"
  }
}

In [12]:
eval_model.visualize_spec()